In [ ]:
from kafka import KafkaConsumer
from json import dump,loads
from s3fs import S3FileSystem
import json
import boto3

In [ ]:
topic_name="your-topic_name" # pass your topic from which your
consumer=KafkaConsumer(
        topic_name,
    
        #pass the consumer_group_id #Note: You should not open same consumer_group at different locations at a time
        group_id="your-group_id",
        
        #pass the bootstrap_server address
        bootstrap_servers=["localhost:9092"],
    
        #latest: read from the last seen offset address #earliest: read from the beginning
        auto_offset_reset="latest", 
    
        #you need to de-serialize when reading from the kafka
        value_deserializer=lambda x:loads(x.decode("utf-8"))
        
        )

I am storing the streaming data into AWS S3 bucket so it can be used for future uses like analyzing using AWS Crawler, etc.

In [ ]:
#As the AWS S3 bucket names are globally unique there is no need to pass aws_access_key_id, aws_secret_access_key
s3 = boto3.resource(
    's3',
    #region_name='us-east-1',
    #aws_access_key_id=KEY_ID,
    #aws_secret_access_key=ACCESS_KEY
)

#It's time to write data into S3 Bucket
#NOTE: This loop runs untill interrupted externally
for msg in consumer:
    
    print(msg.key,msg.value)
    
    content=msg.value
    
    #I'm using the time stamp value as my json file name and then writing into bucket
    s3.Object('your-bucket-name', "{}.json".format(msg.key.decode('utf8', 'strict'))).put(Body=json.dumps(content))
